In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import keras
print(keras.__version__)
# Any results you write to the current directory are saved as output.

In [ ]:
HW = 'humpback-whale-identification'
# TRAIN = '../input/humpback-whale-identification/train/'
TRAIN_CROPPED = "whales-cropped/cropped_train/cropped_train/"
TRAIN_CROPPED_IN = '../input/' + TRAIN_CROPPED

# TEST = '../input/humpback-whale-identification/test/'
TEST_CROPPED = "whales-cropped/cropped_test/cropped_test/"
TEST_CROPPED_IN = '../input/' + TEST_CROPPED

LABELS = '../input/humpback-whale-identification/train.csv'
SAMPLE_SUB = '../input/humpback-whale-identification/sample_submission.csv'

train = pd.read_csv(LABELS)
print("With new_whale:")
train.head()

In [ ]:
MODEL_F = 'Model_InceptionResNetV2.h5'
WEIGHTS_F = 'Weights_InceptionResNetV2.h5'
MODEL = '../input/InceptionResNetV2/'+ MODEL_F
WEIGHTS = '../input/InceptionResNetV2/'+ WEIGHTS_F

In [ ]:
train.describe()

In [ ]:
import random 
from IPython.display import Image
print("Example whale image")

#show sample image
name = random.choice(train['Image'])
print(name)
Image(filename = TRAIN_CROPPED_IN + name)

In [ ]:
criteria = train['Id'] != 'new_whale'
whales_train = train[criteria]
    
print("Without new_whale:")
whales_train.head()

In [ ]:
unique_labels = np.unique(whales_train.Id.values)
labels_list = unique_labels

In [ ]:
whales_train.describe()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# from keras.applications.imagenet_utils import preprocess_input
# from keras.applications.resnet50 import ResNet50, preprocess_input
# from keras.applications.xception import Xception, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

from keras.losses import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, GlobalAveragePooling2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model
from keras.metrics import top_k_categorical_accuracy

import keras.backend as K
from keras.models import Sequential
from PIL import Image
import gc
import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

%matplotlib inline

In [ ]:
IMAGE_HEIGHT = 128
IMAGE_WIDTH = 128
IMAGE_SHAPE = (IMAGE_HEIGHT, IMAGE_WIDTH, 3)

In [ ]:
CLASSES = 5004
EPOCHS = 5
BATCH_SIZE = 32

def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

# setup model
base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = IMAGE_SHAPE)

x = base_model.output
x = GlobalAveragePooling2D(name='avg_pool')(x)
x = Dropout(0.4)(x)
predictions = Dense(CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)
   
# transfer learning
for layer in base_model.layers:
  layer.trainable = True
      
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy', 'mae', top_5_accuracy])

model.summary()

In [ ]:
gc.collect()

# Load model and weights from disc <a name="flow"></a>
____

In [ ]:
# from keras.models import load_model

# # returns a compiled model
# # identical to the previous cell
# model = load_model(MODEL)
# print("Loaded model architecture from disk")
# gc.collect()

# model.load_weights(WEIGHTS)
# print("Loaded model weights from disk")
# model.summary()

# gc.collect()

# Train with ImageDataGenerator and flow_from_dataframe
____

In [ ]:
from keras.callbacks import LambdaCallback, ModelCheckpoint

ROTATE = 20
SEED = 28
BATCH_SIZE = 100

gc.collect()

batch_gc_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: gc.collect())

checkpointer = ModelCheckpoint(filepath='weights.hdf5', 
                               verbose=1, save_best_only=True)

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rescale=1./255,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=whales_train, 
    subset = "training",
    directory=TRAIN_CROPPED_IN, 
    x_col="Image", 
    y_col="Id", 
    has_ext=True, 
    seed = SEED,
    color_mode= "rgb",
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH), 
    batch_size=BATCH_SIZE, 
    class_mode='categorical')

gc.collect()

# Visualize augmented data <a name="flow"></a>
____

In [ ]:
from skimage.io import imread
import PIL.Image as pimage

def plot_images(images_names, path):
    fig, m_axs = plt.subplots(1, len(images), figsize = (20, 10))
    #show the images and label them
    for ii, c_ax in enumerate(m_axs):
        img = imread(os.path.join(path,images_names[ii][0]))
        c_ax.imshow(img)
        c_ax.set_title(images_names[ii][1])

def plot_loaded_images(images_loaded, labels):
    fig, m_axs = plt.subplots(1, len(images_loaded), figsize = (20, 10))
    #show the images and label them
    for ii, c_ax in enumerate(m_axs):
        img = pimage.fromarray(images_loaded[ii], "RGB")
        c_ax.imshow((images_loaded[ii] + 1) / 2)
#         c_ax.set_title(labels[ii])
        
#get the first 5 whale images
images = [(whale_img, whale_label) for (whale_img, whale_label) in zip(whales_train.Image[:5], whales_train.Id[:5])]
plot_images(images, TRAIN_CROPPED_IN)

x_batch, y_batch = next(train_generator)
plot_loaded_images(x_batch[:5], y_batch[:5])

# Learning rate finder <a name="flow"></a>
____

In [ ]:
import keras.backend as K
from keras.callbacks import Callback


class LRFinder(Callback):
    '''
    A simple callback for finding the optimal learning rate range for your model + dataset.

    # Usage
        ```python
            lr_finder = LRFinder(min_lr=1e-5,
                                 max_lr=1e-2,
                                 steps_per_epoch=np.ceil(epoch_size/batch_size),
                                 epochs=3)
            model.fit(X_train, Y_train, callbacks=[lr_finder])

            lr_finder.plot_loss()
        ```

    # Arguments
        min_lr: The lower bound of the learning rate range for the experiment.
        max_lr: The upper bound of the learning rate range for the experiment.
        steps_per_epoch: Number of mini-batches in the dataset. Calculated as `np.ceil(epoch_size/batch_size)`.
        epochs: Number of epochs to run experiment. Usually between 2 and 4 epochs is sufficient.

    # References
        Blog post: jeremyjordan.me/nn-learning-rate
        Original paper: https://arxiv.org/abs/1506.01186
    '''

    def __init__(self, min_lr=1e-5, max_lr=1e-1, steps_per_epoch=None, epochs=None, beta=.98):
        super().__init__()

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.total_iterations = steps_per_epoch * epochs
        self.iteration = 0
        self.history = {}
        self.beta = beta
        self.lr_mult = (max_lr/min_lr)**(1/steps_per_epoch)
        
    def clr(self):
        '''Calculate the learning rate.'''
        x = self.iteration / self.total_iterations
        return self.min_lr * (self.lr_mult**self.iteration)

    def on_train_begin(self, logs=None):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        self.best_loss = 1e9
        self.avg_loss = 0
        self.losses, self.smoothed_losses, self.lrs, self.iterations = [], [], [], []
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.min_lr)

    def on_epoch_end(self, epoch, logs=None):
        print(" Epoch ended learning rate: " + str(K.get_value(self.model.optimizer.lr)))
        
    def on_epoch_begin(self, epoch, logs=None):
        print(" Epoch start learning rate: " + str(K.get_value(self.model.optimizer.lr)))
        
    def on_batch_end(self, epoch, logs=None):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.iteration += 1
        
        #addition
        loss = logs.get('loss')
        self.avg_loss = self.beta * self.avg_loss + (1 - self.beta) * loss
        smoothed_loss = self.avg_loss / (1 - self.beta**self.iteration)
        # Check if the loss is not exploding
        if self.iteration>1 and smoothed_loss > self.best_loss * 1.5:
            self.model.stop_training = True
            return
        if smoothed_loss < self.best_loss or self.iteration==1:
            self.best_loss = smoothed_loss
        
        lr = self.clr()
        self.losses.append(loss)
        self.smoothed_losses.append(smoothed_loss)
        self.lrs.append(lr)
        self.iterations.append(self.iteration)
        
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.iteration)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        K.set_value(self.model.optimizer.lr,lr)

#     def plot_lr(self):
#         '''Helper function to quickly inspect the learning rate schedule.'''
#         plt.plot(self.history['iterations'], self.history['lr'])
#         plt.yscale('log')
#         plt.xlabel('Iteration')
#         plt.ylabel('Learning rate')
#         plt.show()

    def plot_lr(self):
        plt.xlabel('Iterations')
        plt.ylabel('Learning rate')
        plt.plot(self.iterations, self.lrs)
        plt.yscale('log')
        plt.show()

    def plot_loss(self):
        '''Helper function to quickly observe the learning rate experiment results.'''
        plt.plot(self.history['lr'], self.history['loss'])
        plt.xscale('log')
        plt.xlabel('Learning rate')
        plt.ylabel('Loss')
        plt.show()
    
    def plot(self, n_skip=10):
        plt.ylabel('Loss')
        plt.xlabel('Learning rate (log scale)')
        plt.plot(self.lrs[n_skip:-5], self.losses[n_skip:-5])
        plt.xscale('log')
        
    def plot_loss2(self):
        plt.ylabel('Losses')
        plt.xlabel('Iterations')
        plt.plot(self.iterations[10:], self.losses[10:])
        
    def plot_smoothed_loss(self, n_skip=6):
        plt.ylabel('Smoothed Losses')
        plt.xlabel('Learning rate (log scale)')
        plt.plot(self.lrs[n_skip:-5], self.smoothed_losses[n_skip:-5])
        plt.xscale('log')
        
    def plot_loss_change(self, sma=1, n_skip=20, y_lim=(-0.01,0.01), should_lim_y = False):
        """
        Plots rate of change of the loss function.
        Parameters:
            sched - learning rate scheduler, an instance of LR_Finder class.
            sma - number of batches for simple moving average to smooth out the curve.
            n_skip - number of batches to skip on the left.
            y_lim - limits for the y axis.
        """
        derivatives = [0] * (sma + 1)
        for i in range(1 + sma, len(self.lrs)):
            derivative = (self.losses[i] - self.losses[i - sma]) / sma
            derivatives.append(derivative)

        plt.ylabel("d/loss")
        plt.xlabel("learning rate (log scale)")
        plt.plot(self.lrs[n_skip:], derivatives[n_skip:])
        plt.xscale('log')
        if should_lim_y:
            plt.ylim(y_lim)

# Train <a name="flow"></a>
____

In [ ]:
# fits the model on batches with real-time data augmentation:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size

lr_finder = LRFinder(min_lr=1e-6,
                     max_lr=1,
                     steps_per_epoch=np.ceil(train_generator.n//train_generator.batch_size),
                     epochs=EPOCHS)

# Train the loaded model for more epochs
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=STEP_SIZE_TRAIN,
                              epochs=EPOCHS,
                              callbacks=[batch_gc_callback, lr_finder])

In [ ]:
lr_finder.plot_loss()

In [ ]:
lr_finder.plot_smoothed_loss()

In [ ]:
lr_finder.plot_lr()

In [ ]:
lr_finder.plot_loss_change(sma=50)

In [ ]:
lr_finder.plot_loss_change(sma=50, y_lim=(-0.02,0.01), should_lim_y=True)

# Plot train results <a name="flow"></a>
____

In [ ]:
def plot_accuracy(history, should_plot_val = False, should_plot_top5 = False):
    acc = history.history['acc']
    l1 = plt.plot(acc, label='acc')
    
    if should_plot_val:
        val_acc = history.history['val_acc']
        l2 = plt.plot(val_acc, label='val_acc')
    
    if should_plot_top5:
        top5_acc = history.history['top_5_accuracy']
        l3 = plt.plot(top5_acc, label='top_5_accuracy')
        
    plt.legend(loc=2, fontsize="small")
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.show()
    
def plot_loss(history, should_plot_val = False):
    loss = history.history['loss']
    l1 = plt.plot(loss, label='loss')
    
    if should_plot_val:
        val_loss = history.history['val_loss']
        plt.plot(val_loss, label='val_loss')
        
    plt.legend(loc=2, fontsize="small")
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()
    


In [ ]:
plot_accuracy(history, False, True)
plot_loss(history, False)

In [ ]:
model.save(MODEL_F)
print("Saved model architecture to disk")
model.save_weights(WEIGHTS_F)
print("Saved model weights to disk")